In [5]:

!pip install ultralytics roboflow torch matplotlib opencv-python 


Defaulting to user installation because normal site-packages is not writeable


In [6]:

from ultralytics import YOLO
from roboflow import Roboflow
import matplotlib.pyplot as plt
import os

# Download dataset from Roboflow
rf = Roboflow(api_key="A2wycrFsDr1E3JqnNRlT")
project = rf.workspace("nur-byq0f").project("palm-detection-4qh3m")
version = project.version(2)
dataset = version.download("yolov8")

# Print dataset location to verify
print(f"Dataset downloaded to: {dataset.location}")



loading Roboflow workspace...
loading Roboflow project...
Dataset downloaded to: c:\Users\balak\PalmTreeDet\Palm-Detection-2


In [7]:
import os
from ultralytics import YOLO

# Memory optimization
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Load model (using 's' version for better accuracy than 'n' with minimal memory impact)
model = YOLO("yolov8s.pt") 

results = model.train(
    # --- Core Training ---
    data=os.path.join(dataset.location, "data.yaml"),
    epochs=20,
    imgsz=512,        # Optimal for palm tree detection (balance between detail and memory)
    batch=6,          # Stable batch size for 4GB GPU
    device="0",
    amp=True,         # Mixed precision training (critical for 4GB GPUs)
    patience=20,      # Longer patience for palm detection
    
    # --- Palm-Specific Augmentations ---
    hsv_h=0.015,      # Slight hue variation (palms have consistent colors)
    hsv_s=0.4,        # Moderate saturation changes
    hsv_v=0.3,        # Limited value changes (avoid dark/bright extremes)
    fliplr=0.5,       # Keep horizontal flips (symmetrical trees)
    degrees=7.0,      # Small rotations (trees mostly upright)
    perspective=0.001, # Minimal perspective (aerial view consistency)
    
    # --- Memory Optimizations ---
    cache="ram",      # Faster if you have >16GB system RAM
    workers=2,        # Reduce if RAM issues occur
    optimizer="AdamW", # More memory efficient than SGD
    
    # --- Palm Detection Enhancements ---
    mixup=0.05,       # Helps with overlapping trees
    copy_paste=0.05,  # Simulates occluded palms
    overlap_mask=True, # Better for dense palm clusters
    mask_ratio=2,     # Optimizes mask processing
)

Ultralytics 8.3.96  Python-3.9.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=c:\Users\balak\PalmTreeDet\Palm-Detection-2\data.yaml, epochs=20, time=None, patience=20, batch=6, imgsz=512, save=True, save_period=-1, cache=ram, device=0, workers=2, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=2, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, 

train: Scanning C:\Users\balak\PalmTreeDet\Palm-Detection-2\train\labels.cache... 8532 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8532/8532 [00:00<?, ?it/s]


train: 9.4GB RAM required to cache images with 50% safety margin but only 0.9/15.4GB available, not caching images 


val: Scanning C:\Users\balak\PalmTreeDet\Palm-Detection-2\valid\labels.cache... 813 images, 0 backgrounds, 0 corrupt: 100%|██████████| 813/813 [00:00<?, ?it/s]

val: 0.9GB RAM required to cache images with 50% safety margin but only 0.6/15.4GB available, not caching images 


...

      20/20       1.8G      1.222     0.8247      1.588         23        512: 100%|██████████| 1422/1422 [03:05<00:00,  7.68it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 68/68 [00:09<00:00,  7.17it/s]

                   all        813       1951      0.832      0.869       0.89      0.538



20 epochs completed in 1.123 hours.
Optimizer stripped from runs\detect\train6\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train6\weights\best.pt, 22.5MB

Validating runs\detect\train6\weights\best.pt...
Ultralytics 8.3.96  Python-3.9.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 68/68 [00:09<00:00,  6.94it/s]


                   all        813       1951      0.833      0.868       0.89      0.538
Speed: 0.3ms preprocess, 4.3ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs\detect\train6


In [ ]:
# 3. Enhanced Validation
metrics = model.val(
    conf=0.25,              # Confidence threshold
    iou=0.45,               # NMS IoU threshold
    plots=True,             # Generate plots
    save_json=True ,         # Save metrics to JSON
    visualize =True #show modified samples
)

# 4. Model Export (optional)
model.export(format='onnx')  # For production deployment

Ultralytics 8.3.96  Python-3.9.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning C:\Users\balak\PalmTreeDet\Palm-Detection-2\valid\labels.cache... 813 images, 0 backgrounds, 0 corrupt: 100%|██████████| 813/813 [00:00<?, ?it/s]

WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



val: Caching images (0.6GB RAM): 100%|██████████| 813/813 [00:01<00:00, 517.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 136/136 [00:10<00:00, 12.37it/s]


                   all        813       1951       0.88      0.803      0.852      0.548
Speed: 0.4ms preprocess, 7.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Saving runs\detect\train62\predictions.json...
Results saved to runs\detect\train62
Ultralytics 8.3.96  Python-3.9.7 torch-2.5.1+cu121 CPU (AMD Ryzen 7 4800H with Radeon Graphics)

PyTorch: starting from 'runs\detect\train6\weights\best.pt' with input shape (1, 3, 512, 512) BCHW and output shape(s) (1, 5, 5376) (21.5 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 14.5/14.5 MB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 226.4/226.4 MB 1.2 MB/s eta 0:00:00

requirements: AutoUpdate success  239.2s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu']
requirements:  Restart runtime or 

'runs\\detect\\train6\\weights\\best.onnx'

: 